In [1]:
# Module Importations
from scipy.stats import reciprocal
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import keras
import numpy as np
import time

print('keras.__version__=', keras.__version__)

Using TensorFlow backend.
keras.__version__= 2.3.1


In [8]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.models import keras_helpers
from src.models import model_evaluation

In [9]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [10]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_test(original_dataset, 0.8)

In [11]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)
evaluation_set = evaluation_set.drop("profile_id", axis = 1)

In [12]:
# Create targets dataset (Stator yoke, tooth and winding temperature)

# Training Set
stator_training_data = training_set.drop(["stator_yoke", "stator_tooth", "stator_winding"], axis = 1)

stator_label_data = training_set[["stator_yoke", "stator_tooth", "stator_winding"]].copy()

# Evaluation Set
stator_eval_data = evaluation_set.drop(["stator_yoke", "stator_tooth", "stator_winding"], axis = 1)

stator_eval_label = evaluation_set[["stator_yoke", "stator_tooth", "stator_winding"]].copy()

In [13]:
# Create train & test arrays
X_train, X_test, y_train, y_test = train_test_split(stator_training_data, stator_label_data, test_size = 0.2, random_state = 0)

print("X_train:", X_train.shape, "Y_train:", y_train.shape)

X_train: (159691, 9) Y_train: (159691, 3)


In [8]:
# Multilayer Perceptron (Target - Stator Temperatures)

# Clear existing models
keras.backend.clear_session()

# Define parameter distribution
param_distribs = {
    "n_hidden": [1, 4, 8],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
    }

#    "input_shape": [9],
#    "output_shape": [3]

# Build model
wrapped_model = keras_helpers.wrap_model()

# Initialise random search
rnd_search_cv = RandomizedSearchCV(wrapped_model, param_distribs, n_iter = 10, cv = 3)

# Name model
model_type = "MLP-StatorTemps"
model_id = keras_helpers.name_model(model_type)
filepath_full = keras_helpers.make_save_string(model_id)

# Set save & early stop callbacks
earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

# Train model
rnd_search_cv.fit( X_train, y_train, epochs = 50, validation_data =(X_test, y_test), callbacks =[checkpoint_cb, earlystop_cb])

NameError: name 'keras_helpers' is not defined

In [15]:
# Model Evaluation

# Load model
filepath = r'C:\Developer\electric_motor_thermal_modelling\Models\YC_MLP-StatorTemps_2020_08_18-11_35_56.h5'

model = keras.models.load_model(filepath)

# Evaluation
start_time = time.process_time()
stator_pred_eval = model.predict(stator_eval_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_name = "MLP - Stator"
model_evaluation.evaluate_model(model_name, stator_eval_label, stator_pred_eval)
#model.evaluate(stator_test_data, stator_test_label)

Prediction Time: 14.265625
MLP - Stator rmse (Eval): 0.22798583686673063
MLP - Stator mae (Eval): 0.14750733649272577
MLP - Stator r2 (Eval): 0.9479234362492818
